# Using SDK to deploy a Python model with multiple fields

In [ ]:
BUNDLE_WORKING_DIR = "/tmp/bundle_multiple_fields_working_dir"
BUNDLE_ZIP_FILE = "/tmp/mymodel_multiple_fields.zip"

!mkdir -p $BUNDLE_WORKING_DIR

## Create `predict.py`

In [94]:
%%writefile $BUNDLE_WORKING_DIR/predict.py

import time

def init(bundle_path: str):
    """
    One-time initialization here.  For example, loading a serialized model from disk.
    
    :param bundle_path: Path to unzipped bundle, used to construct file path to bundle contents, i.e. os.path.join(bundle_path, "model.pkl")
    """
    print(bundle_path)
    time.sleep(1)
    
def predict(model_input: dict) -> dict: 
    """
    Generate prediction.

    Must return a dict, representing model output. Returned values must be of one of the following types:
      * Any numbers.Number (float, int, etc.)
      * str
      * list or ndarray of str
      * list or ndarray of numbers.Number    
    
    :param model_input: a dict containing model input
    :return: model output dict
    """
    if 'a' not in model_input:
        raise ValueError("Input must contain the key 'a'")
    
    if 'b' not in model_input:
        raise ValueError("Input must contain the key 'b'")
    
    a = float(model_input['a'])
    b = float(model_input['b'])
    
    return {
        "product": a * b,
        "sum": a + b
    }    

Overwriting /tmp/bundle_multiple_fields_working_dir/predict.py


## Create bundle ZIP file

In [95]:
!cd $BUNDLE_WORKING_DIR; zip -r $BUNDLE_ZIP_FILE .

updating: predict.py (deflated 55%)


## Test model locally

In [87]:
from lucidworks.ml.sdk import LocalBundleRunner
runner = LocalBundleRunner(BUNDLE_ZIP_FILE)

/var/folders/dd/rflrpdn974gchxm12nnn2qmh0000gn/T/tmpywz3zi13


In [88]:
runner.predict({
    "a": 5,
    "b": 5
})

{'product': 25.0, 'sum': 10.0}

## Deploy model to Fusion

In [96]:
from lucidworks.ml.sdk import MLServiceSDKFusionClient
from requests.auth import HTTPBasicAuth

model_id = 'arithmetic'
app_name = 'ml_test'

client = MLServiceSDKFusionClient('http://localhost:6764/api', 
                                  app_name,
                                  auth=HTTPBasicAuth('admin', 'password123'))
client.upload_model(BUNDLE_ZIP_FILE, model_id)

In [90]:
client.list_models()

[{'model_id': 'echo',
  'type': 'python',
  'last_modified': '2019-08-19T14:25:22Z'},
 {'model_id': 'opennlp',
  'type': 'open-nlp',
  'last_modified': '2019-08-19T14:44:34Z'},
 {'model_id': 'arithmetic',
  'type': 'python',
  'last_modified': '2019-08-19T19:29:52Z'},
 {'model_id': 'spacy',
  'type': 'python',
  'last_modified': '2019-08-19T14:44:33Z'}]

## Test model prediction in Fusion

In [107]:
client.predict(model_id, {
    "a": 5,
    "b": 5
})

{'product': 25.0, 'sum': 10.0}

## Generate predictions in Fusion Query Pipeline with custom JS transformation scripts

In [92]:
input_js_script = """
var modelInput = new java.util.HashMap()
var a = request.getFirstParam("a")
var b = request.getFirstParam("b")

modelInput.put("a", a || "1")
modelInput.put("b", b || "1")
modelInput
"""

output_js_script = """
request.putSingleParam("product", modelOutput.get("product"))
request.putSingleParam("sum", modelOutput.get("sum"))
"""
client.create_ml_query_pipeline('sample_arithmetic_ml', 
                                model_id, 
                                input_js_script, 
                                output_js_script)

In [93]:
response = client.query('sample_arithmetic_ml', 'ml_test', '', {
    'a': 5,
    'b': 5
})
print("product:", response['responseHeader']['params']['product'])
print("sum:", response['responseHeader']['params']['sum'])
print("")
print("Response:")
print(json.dumps(response, indent=4))

product: 25
sum: 10

Response:
{
    "response": {
        "numFound": 0,
        "start": 0,
        "maxScore": 0.0,
        "docs": []
    },
    "responseHeader": {
        "zkConnected": true,
        "status": 0,
        "QTime": 2,
        "params": {
            "a": "5",
            "b": "5",
            "product": "25",
            "lw.pipelineId": "sample_arithmetic_ml",
            "fl": [
                "*",
                "score"
            ],
            "start": "0",
            "isFusionQuery": "true",
            "sum": "10",
            "rows": "10",
            "fusionQueryId": "53DwtByBQE",
            "q": "",
            "defType": "edismax",
            "context": "app:ml_test",
            "wt": "json"
        },
        "totalTime": 240
    }
}
